<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 3: Future projections for each impact

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 4 included the
manuscript <<< insert doi >>>. Future risks for fire. We show median changes in
risk with respect to the historic.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import rioxarray
from carbonplan.data import cat


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from showit import image
import scipy as sp
import geopandas as gpd
import regionmask as rm
import altair as alt
alt.themes.enable("carbonplan_light")

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
coarsen = 4

In [ ]:
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

In [ ]:
region_bboxes = {
    "CONUS": {"x": slice(-3e6, 3e6), "y": slice(4e6, 0)},
    "PNW": {"x": slice(-2.5e6, -1e6), "y": slice(3.5e6, 2.4e6)},
    "Southwest": {"x": slice(-1.8e6, 0.9e6), "y": slice(1.8e6, 0.9e6)},
    "Midwest": {"x": slice(-0.7e6, 0.5e6), "y": slice(3e6, 1.2e6)},
    "California": {"x": slice(-2.3e6, -1.8e6), "y": slice(2.5e6, 1.2e6)},
    "Southeast": {
        "x": slice(0.6e6, 1.8e6),
        "y": slice(1.6e6, 0.3e6),
    },
}

# Load in fire data


In [ ]:
historical_fire = xr.open_zarr(
    get_store(
        "carbonplan-forests", "risks/results/paper/fire_terraclimate_v6.zarr"
    )
).load()
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds = (
    xr.open_zarr(
        get_store("carbonplan-forests", "risks/results/paper/fire_cmip_v6.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
historical_annual = (
    xr.open_zarr(
        get_store(
            "carbonplan-forests",
            "risks/results/paper/fire_terraclimate_v6.zarr",
        )
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
results_dict = {"fire": {}, "insects": {}, "drought": {}}
for region, bbox in region_bboxes.items():
    results_dict["fire"][region] = {}
    selected = ds.sel(**region_bboxes[region])
    selected_historical = historical_annual.sel(**region_bboxes[region])
    results_dict["fire"][region]["future"] = (
        selected.mean(dim=["x", "y"]).rolling(dim={"year": 10}).mean().compute()
    )
    results_dict["fire"][region]["historical"] = (
        selected_historical.mean(dim=["x", "y"])
        .rolling(dim={"year": 10})
        .mean()
        .compute()
    )

In [ ]:
gcms = [
    ("ACCESS-CM2", "r1i1p1f1"),
    ("ACCESS-ESM1-5", "r10i1p1f1"),
    ("MRI-ESM2-0", "r1i1p1f1"),
    ("MIROC-ES2L", "r1i1p1f2"),
    ("MPI-ESM1-2-LR", "r10i1p1f1"),
    ("CanESM5-CanOE", "r3i1p2f1"),
]

### Load in insect and drought data


In [ ]:
for impact in ["insects", "drought"]:
    historical_ds = (
        xr.open_zarr(
            get_store(
                "carbonplan-forests",
                "risks/results/paper/{}_terraclimate.zarr".format(impact),
            )
        )
        .assign_coords({"year": np.arange(1995, 2020, 10)})
        .compute()
    )
    future_ds = (
        xr.open_zarr(
            get_store(
                "carbonplan-forests",
                "risks/results/paper/{}_cmip_v5.zarr".format(impact),
            )
        )
        .assign_coords({"year": np.arange(1975, 2100, 10)})
        .compute()
    )

    for region, bbox in region_bboxes.items():
        results_dict[impact][region] = {}
        selected_future = future_ds.sel(**region_bboxes[region])
        selected_historical = historical_ds.sel(**region_bboxes[region])
        results_dict[impact][region]["future"] = selected_future.mean(
            dim=["x", "y"]
        ).compute()
        results_dict[impact][region]["historical"] = selected_historical.mean(
            dim=["x", "y"]
        ).compute()

## Plot the figure


In [ ]:
gcms = [
    ("MRI-ESM2-0", (0, 0)),
    ("MIROC-ES2L", (1, 0)),
    ("MPI-ESM1-2-LR", (2, 0)),
    ("ACCESS-ESM1-5", (3, 0)),
    ("ACCESS-CM2", (4, 0)),
    ("CanESM5-CanOE", (5, 0)),
]
scenario_colors = {
    "ssp245": "#59A82F",
    "ssp370": "#D8B525",
    "ssp585": "#D83232",
}
scenario_colors_light = {
    "ssp245": "#DEEED5",
    "ssp370": "#F7F0D3",
    "ssp585": "#F7D6D6",
}
titles = {
    "fire": "Burn area\n(fraction/year)",
    "drought": "Drought-related\nmortality (%/year)",
    "insects": "Insect-related\nmortality (%/year)",
}
ylims = {
    "fire": (0, 0.05),
    "drought": (0, 2.5),
    "insects": (0, 1),
}
ssp_rename = {"ssp245": "SSP2-4.5", "ssp370": "SSP3-7.0", "ssp585": "SSP5-8.5"}

In [ ]:
plt.rcParams.update({"font.size": 14, "svg.fonttype": "none"})

fig, axarr = plt.subplots(
    nrows=3,
    ncols=len(region_bboxes),
    figsize=(24, 10),
    sharex=True,
    sharey="row",
)

## Fire
for j, region in enumerate(region_bboxes.keys()):
    for scenario in ["ssp245", "ssp370", "ssp585"]:
        for (gcm, location) in gcms:
            results_dict["fire"][region]["future"].probability.sel(
                gcm=gcm, scenario=scenario
            ).sel(year=slice("1970", "2099")).plot(
                ax=axarr[0, j], color=scenario_colors_light[scenario], alpha=1
            )

    for scenario in ["ssp245", "ssp370", "ssp585"]:
        results_dict["fire"][region]["future"].probability.sel(
            scenario=scenario
        ).mean(dim="gcm").plot(
            ax=axarr[0, j],
            color=scenario_colors[scenario],
            label=ssp_rename[scenario],
        )
    results_dict["fire"][region]["historical"].historical.plot(
        ax=axarr[0, j], color="k", linestyle="-", label="Historical"
    )

    # Drought and insects
    for i, impact in enumerate(["drought", "insects"]):
        results_dict[impact][region]["historical"].probability.plot(
            ax=axarr[i + 1, j], color="k", zorder=50
        )

        for scenario in ["ssp245", "ssp370", "ssp585"]:
            for (gcm, location) in gcms:
                results_dict[impact][region]["future"].probability.sel(
                    scenario=scenario, gcm=gcm
                ).sel(year=slice(1970, 2099)).plot(
                    ax=axarr[i + 1, j],
                    color=scenario_colors_light[scenario],
                )

        for scenario in ["ssp245", "ssp370", "ssp585"]:
            results_dict[impact][region]["future"].probability.sel(
                scenario=scenario
            ).mean(dim="gcm").plot(
                ax=axarr[i + 1, j],
                color=scenario_colors[scenario],
                label=ssp_rename[scenario],
            )

    for i, (impact, title) in enumerate(titles.items()):
        axarr[i, j].set_xlim(1970, 2100)
        axarr[i, j].set_xlabel("")
        axarr[i, j].set_ylabel("")
        axarr[i, j].set_ylim(ylims[impact])
        axarr[i, j].set_title("")
for i, (impact, title) in enumerate(titles.items()):
    axarr[i, 0].set_ylabel(title)
for j, region in enumerate(region_bboxes):
    axarr[0, j].set_title(region)
axarr[0, 5].legend()
plt.tight_layout()
for format_string in ["svg", "png"]:
    plt.savefig("fig4_v17." + format_string, format=format_string)